In [2]:
import os
import pandas as pd
import numpy as np
import re

from itables import show

# Dados do INEP, IBGE e VIS Data 3


_O conceito técnico de abandono é diferente de evasão. Abandono quer dizer que o aluno deixa a escola em um ano mas retorna no ano seguinte. **Evasão significa que o aluno sai da escola e não volta mais para o sistema.**_


Para possibilitar a análise desejada serão agrupados dados dos [Indicadores Educacionais do INEP](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais), da [API de dados agregados do IBGE](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-Variaveis-agregadosAgregadoPeriodosPeriodosVariaveisVariavelGet) e dados do Programa Bolsa Família fornecidos pela [VIS Data 3](https://aplicacoes.cidadania.gov.br/vis/data3/v.php?q[]=r6JtZJCug7BtxKW25rV%2FfmdhhJFkl21kmK19ZnB1ZW6maX7KmZO20qfOnJm%2B6IianbSon7SfrrqqkpKcmcuppsK2iKextVi1mpyuwZxNzsmY2F1zyuDAk522pHa2YH9%2BaV6EkmOXbWSEm8GcobZVetufrL%2BrkbbHlNddmMnuslSqvaGmmZ67sliqkseU1rCYmOGuoK%2BtcHXfmrnBnGiS1KjXYK5%2B3q6noWisot6nbY6kksrAlNiscZqif2Rue2JqrGZ9f15Ny8mY2F1zv%2BGspbCslKDapm2zo6C8gaHfqZ994LuYXcVwoNqlwLNyk7jNps94bsPcuaehg3Ct7qZwyViQuNSYirSbwultdKmtqJnap7yKdFSJkWWbamSNqH1lY2ipot6nbY6Zk7bXn4qin9DgbaKxtKFa3qexb7RovcKf3aJuw9y5p6GDcKDapcCzcmjK1qCNuFTA3MCZXL%2Bdn%2BdZjbucoLbCodl7cIStfWZvdWVtpml%2BdVehv8ahin2Vw9rDoFytoa3eWbvDo5l3xqHOXrCY4a6gr61woNqlwLNyaL3Cn92ibpjuwqFfw1ad2qyybq6VvM9TqqqY0NquoquEcmGraX9%2FZF6HjmObZFPR47KiXHCYm%2ByebcWfksWBc8yjks7vsZOiqaJ4qVnBtpybd9Oi36uXhbuvmpu%2BoXSzp8K7nJ%2FAxGKqn5m87MGYm66Wp6Vrdm6cmcrGU9iyn8mbsqKgcVWf5ayybqWiw81Tz6uXfviImp20qJ%2B0n666qpKSnJnLqabCtoinsbVYtZqcrsGcTc7JmNhdc8rgwJOdtqR4tmB%2FfmlghJFml21khJvBnKG2VWLcmsCzV6S%2FxqGKfamPsYFnenhVruGeu26pnMzPl5J9lcPaw6B2gqOv5p6%2Ft5pcl9dloHFmia12VKG0qJ%2BZp8K6o028z5eTXZjJ7rJUqr2hppmeu7JYqpLHlNawmJjhrqCvrXB135q5wZxoktSo17l5vugQ4aixlq2Ze7K8nJPAxJwt3qXG3MBXgqmi%2FSaltq%2BqTaejeYpllNE%2B9lSLvalpq2l%2Ff2BQncKgLeqfxtzAVIyKe1qhmm2%2BmJ%2FLyqWKoZh9yK6ma3plbKxicKSYmcbTU9yio77uwJWgt1X9GaxttJiaGg6f056mfcuPelxwlq484m2drKGGk2OcblyA0a6gq7pVrN6prsGqjrvQUy3dpn3hrqH%2F9aGj2qxtnnlzd4mUiq2Uz%2B%2B2plysmlrGmr99aV2JlFyNk5TJ6r9UoLdVfN6nsrT62rrKooqq9gbftqNccJauPOJtnayhhpNjnG5cgNGuoKu6VZ7oWY%2BzpZK9JODNpqJ96BDdoLGkWqGabb6Yn8vKpYqhmH3IrqZremVsrGLJvnKp091lmm1miqt%2BYWx5iWqpc31%2BcV2Hu24%3D&ma=ano&dt1=2013-01-01&dt2=2021-01-01&ag=m&ultdisp=1&ultdisp=0).

In [5]:
# Constants
INTERIM_DATA_DIR = "../data/interim"
PROCESSED_DATA_DIR = "../data/processed"
PRIMARY_KEY = "CO_MUNICIPIO"

COMMOM_COLUMNS = [
    "NU_ANO_CENSO",
    "NO_REGIAO",
    "SG_UF",
    "CO_MUNICIPIO",
    "NO_MUNICIPIO",
    "NO_CATEGORIA",
    "NO_DEPENDENCIA",
]
DROP_COLUMNS_SET = {
    "NU_ANO_CENSO",
    "NO_REGIAO",
    "SG_UF",
    "NO_MUNICIPIO",
    "MED_NS_CAT_0",
    "CO_UF",
    # "ano",
}
MERGE_COLUMNS = ["CO_MUNICIPIO", "NO_CATEGORIA", "NO_DEPENDENCIA"]
state_code_map = {
    "AC": 12,
    "AL": 27,
    "AM": 13,
    "AP": 16,
    "BA": 29,
    "CE": 23,
    "DF": 53,
    "ES": 32,
    "GO": 52,
    "MA": 21,
    "MG": 31,
    "MS": 50,
    "MT": 51,
    "PA": 15,
    "PB": 25,
    "PE": 26,
    "PI": 22,
    "PR": 41,
    "RJ": 33,
    "RN": 24,
    "RO": 11,
    "RR": 14,
    "RS": 43,
    "SC": 42,
    "SE": 28,
    "SP": 35,
    "TO": 17,
}

In [6]:
# Auxiliar functions
OBJECT_COLUMNS = [*COMMOM_COLUMNS, "NO_UF"]


def _cast_columns_to_float(df):
    columns_to_convert = df.select_dtypes(include="object").columns
    columns_to_convert = [
        col for col in columns_to_convert if col not in OBJECT_COLUMNS
    ]
    try:
        df[columns_to_convert] = df[columns_to_convert].astype(float)
    except ValueError as e:
        print(f"\n\nError converting columns to float: {e}")
        print(f"Columns: {columns_to_convert}\n\n")
    return df


def _add_prefix_header(df, prefix):
    columns_name_map = {}
    columns = df.columns
    for col in columns:
        if col in COMMOM_COLUMNS:
            columns_name_map[col] = col
        else:
            columns_name_map[col] = f"{prefix}_{col}"
    return df.rename(columns=columns_name_map)


def _get_df_info(df, df_name):
    print(f"------------------------ {df_name} ------------------------")
    df.info()
    print(
        f"\nREGISTROS NÃO REPETIDOS: {df.drop_duplicates().shape[0]} de {df.shape[0]}\n"
    )


def _remove_childhood_columns(df):
    # Remove columns with `INF_` in the name - Childhood Education data
    childhood_education_columns = [col_name for col_name in df if "INF_" in col_name]
    if childhood_education_columns:
        print(
            f"***Removing {len(childhood_education_columns)} columns:",
            ", ".join(childhood_education_columns),
        )
        df.drop(childhood_education_columns, axis=1, inplace=True)


def _extract_year(string):
    match = re.search(r"\b\d{4}(?!\d)", string)
    if match:
        return match.group()
    else:
        return None

In [7]:
# Merge year data
TARGET_DATASET = "TXT_CLEAN"


def _merge_year_data(year_data_dict, external_data_df, year_dataset):
    total_colunas = year_data_dict[TARGET_DATASET].shape[1]

    # Check for NaN values in the TXT_FUN_TX_EVASAO_TOTAL and TXT_MED_TX_EVASAO_TOTAL columns
    print(
        "Número de linhas com TXT_FUN_TX_EVASAO_TOTAL e TXT_MED_TX_EVASAO_TOTAL como NaN:",
        year_data_dict[TARGET_DATASET]
        .query("@pd.isna(TXT_FUN_TX_EVASAO_TOTAL) & @pd.isna(TXT_MED_TX_EVASAO_TOTAL)")
        .shape[0],
    )

    print(
        f">>>{TARGET_DATASET} add {year_data_dict[TARGET_DATASET].shape[1]} new features"
    )

    # Merge data
    merged_data = year_data_dict[TARGET_DATASET]  # use TXT_CLEAN_* as base
    for key, data in year_data_dict.items():
        if key != TARGET_DATASET:
            merge_columns_set = list(set(MERGE_COLUMNS).intersection(data.columns))

            if len(merge_columns_set) < 3:
                print(
                    f"\n!!!!Warning: {key} will be merged with only {merge_columns_set} columns\n"
                )

            print(f"Merging {key}...")
            total_colunas += data.shape[1] - len(merge_columns_set)
            print(f">>>{key} add {data.shape[1] - len(merge_columns_set)} new features")
            merged_data = pd.merge(merged_data, data, on=merge_columns_set, how="left")

    # Add external data
    print(f"Merging external...")
    total_colunas += 2
    print(f">>>external_data add 2 new features")
    current_year = _extract_year(year_dataset)
    external_data_current_year = external_data_df[
        [
            "CO_MUNICIPIO",
            f"PIB_{current_year}",
            f"PERCENTUAL_FAMILIAS_PBF_{current_year}",
        ]
    ]

    # Add column with year of the dataset
    total_colunas += 1
    print(f">>>ANO add 1 new feature")
    merged_data["ANO"] = current_year

    # Add covid column, 1 for 2020 & 2021 and 0 for other years
    # TODO: ver se pode fazer isso mesmo para nosso algoritmo...
    total_colunas += 1
    print(f">>>PANDEMIA_COVID add 1 new feature")
    merged_data["PANDEMIA_COVID"] = 1 if current_year in ["2020", "2021"] else 0

    merged_data = pd.merge(
        merged_data, external_data_current_year, on=PRIMARY_KEY, how="left"
    )

    print(
        f"Finished merging data for {year_dataset}. Total de colunas: {total_colunas}"
    )
    return merged_data

In [5]:
# Build datasets for all years
years = [f"{year}_NCR" for year in range(2013, 2022)]
# years = ["2019_NCR"]
# years = list(range(2013, 2022))

# Tables with information of all years
external_data_df = pd.read_csv(
    os.path.join(INTERIM_DATA_DIR, "external_data.csv"), delimiter=";"
)

for year in years:
    print(f"\n\n---------- Start processing {year}... ----------")

    year_data_dict = {}
    current_year_files = os.listdir(os.path.join(INTERIM_DATA_DIR, year))

    for file in current_year_files:

        file_path = os.path.join(INTERIM_DATA_DIR, year, file)
        print(f"Loading {file_path}")

        dataset_name = file.rstrip(".csv")
        year_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")

        ########################################################
        # Ajustes para o merge
        if TARGET_DATASET in dataset_name:
            year_data_dict[TARGET_DATASET].rename(
                columns={"NO_LOCALIZACAO": "NO_CATEGORIA"}, inplace=True
            )
        ########################################################

        # Remove unnecessary columns common to all datasets
        existing_columns_to_drop = set(DROP_COLUMNS_SET).intersection(
            year_data_dict[dataset_name].columns
        )
        print(
            f"***Removing {len(existing_columns_to_drop)} columns:",
            ", ".join(existing_columns_to_drop),
        )
        year_data_dict[dataset_name].drop(
            existing_columns_to_drop, axis=1, inplace=True
        )
        # Remove columns with `INF_` in the name - Childhood Education data
        _remove_childhood_columns(year_data_dict[dataset_name])
        # Replace , to .
        year_data_dict[dataset_name].replace(
            regex=r",(\d+)", value=r".\1", inplace=True
        )
        # Remove "--" and "***" string from empty cells
        year_data_dict[dataset_name].replace("--", np.nan, inplace=True)
        year_data_dict[dataset_name].replace("***", np.nan, inplace=True)
        # Cast columns to float
        year_data_dict[dataset_name] = _cast_columns_to_float(
            year_data_dict[dataset_name]
        )
        # Rename columns to add table prefix
        prefix = dataset_name.split("_")[0]
        year_data_dict[dataset_name] = _add_prefix_header(
            year_data_dict[dataset_name], prefix
        )

        # Add state code column
        if "TXT" in dataset_name:
            year_data_dict[dataset_name]["TXT_NO_UF"] = year_data_dict[dataset_name][
                "TXT_NO_UF"
            ].map(state_code_map)
            year_data_dict[dataset_name].rename(
                columns={"TXT_NO_UF": "UF_CODE"}, inplace=True
            )

    print(f"\n---------- Start merging {year} data... ----------")
    merged_data = _merge_year_data(year_data_dict, external_data_df, year)

    _get_df_info(merged_data, year)

    # Save data
    print(f"Saving data for {year}...")
    output_dir = os.path.join(PROCESSED_DATA_DIR, year)
    os.makedirs(output_dir, exist_ok=True)
    merged_data.to_csv(
        os.path.join(output_dir, "complete_data.csv"), sep=";", index=False
    )

    # Save sliced data: NO_CATEGORIA==Total and NO_DEPENDENCIA==Total
    total_total_data = merged_data[
        (merged_data["NO_CATEGORIA"] == "Total")
        & (merged_data["NO_DEPENDENCIA"] == "Total")
    ].copy()
    total_total_data.drop(["NO_CATEGORIA", "NO_DEPENDENCIA"], axis=1, inplace=True)

    total_total_data.to_csv(
        os.path.join(output_dir, "total_total_data.csv"), sep=";", index=False
    )

    # Save sliced data: Only data without FUN columns
    total_total_data_med = total_total_data.filter(regex="^(?!.*FUN)")
    total_total_data_med.to_csv(
        os.path.join(output_dir, "total_total_data_med.csv"), sep=";", index=False
    )

    # Save sliced data: Only data without MED columns
    total_total_data_fun = total_total_data.filter(regex="^(?!.*MED)")
    total_total_data_fun.to_csv(
        os.path.join(output_dir, "total_total_data_fun.csv"), sep=";", index=False
    )



---------- Start processing 2013_NCR... ----------
Loading ../data/interim/2013_NCR/TDI_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2013_NCR/AFD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 5 columns: ED_INF_ADEQ_DOC_GRUPO1, ED_INF_ADEQ_DOC_GRUPO2, ED_INF_ADEQ_DOC_GRUPO3, ED_INF_ADEQ_DOC_GRUPO4, ED_INF_ADEQ_DOC_GRUPO5
Loading ../data/interim/2013_NCR/HAD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 1 columns: ED_INF_MEDIA_TOTAL_HORAS_AULA


/tmp/ipykernel_28611/2559436282.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  year_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")


Loading ../data/interim/2013_NCR/IRD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2013_NCR/DSU_CLEAN.csv
***Removing 5 columns: NO_REGIAO, SG_UF, CO_UF, NO_MUNICIPIO, NU_ANO_CENSO
***Removing 1 columns: ED_INF_PERC_DOC_SUPERIOR_TOTAL
Loading ../data/interim/2013_NCR/ATU_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 1 columns: ED_INF_MEDIA_TOTAL_ALUNOS_SALA


/tmp/ipykernel_28611/2559436282.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  year_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")


Loading ../data/interim/2013_NCR/TXT_CLEAN.csv
***Removing 3 columns: NO_MUNICIPIO, NU_ANO_CENSO, NO_REGIAO
Loading ../data/interim/2013_NCR/ICG_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2013_NCR/TXR_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO


...

Loading ../data/interim/2015_NCR/IED_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO

---------- Start merging 2015_NCR data... ----------
Número de linhas com TXT_FUN_TX_EVASAO_TOTAL e TXT_MED_TX_EVASAO_TOTAL como NaN: 10
>>>TXT_CLEAN add 10 new features
Merging TDI_CLEAN...
>>>TDI_CLEAN add 2 new features
Merging AFD_CLEAN...
>>>AFD_CLEAN add 10 new features
Merging HAD_CLEAN...
>>>HAD_CLEAN add 2 new features
Merging IRD_CLEAN...
>>>IRD_CLEAN add 4 new features
Merging DSU_CLEAN...
>>>DSU_CLEAN add 2 new features
Merging ATU_CLEAN...
>>>ATU_CLEAN add 2 new features
Merging ICG_CLEAN...
>>>ICG_CLEAN add 6 new features
Merging TXR_CLEAN...
>>>TXR_CLEAN add 6 new features
Merging IED_CLEAN...
>>>IED_CLEAN add 12 new features
Merging external...
>>>external_data add 2 new features
>>>ANO add 1 new feature
>>>PANDEMIA_COVID add 1 new feature
Finished merging data for 2015_NCR. Total de colunas: 60
------------------------ 2015_NCR ------------------------
<c

/tmp/ipykernel_28611/2559436282.py:23: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  year_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")


Loading ../data/interim/2021_NCR/AFD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 5 columns: ED_INF_ADEQ_DOC_GRUPO1, ED_INF_ADEQ_DOC_GRUPO2, ED_INF_ADEQ_DOC_GRUPO3, ED_INF_ADEQ_DOC_GRUPO4, ED_INF_ADEQ_DOC_GRUPO5
Loading ../data/interim/2021_NCR/HAD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 1 columns: ED_INF_MEDIA_TOTAL_HORAS_AULA


/tmp/ipykernel_28611/2559436282.py:23: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  year_data_dict[dataset_name] = pd.read_csv(file_path, delimiter=";")


Loading ../data/interim/2021_NCR/IRD_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2021_NCR/DSU_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 1 columns: ED_INF_PERC_DOC_SUPERIOR_TOTAL
Loading ../data/interim/2021_NCR/ATU_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
***Removing 1 columns: ED_INF_MEDIA_TOTAL_ALUNOS_SALA
Loading ../data/interim/2021_NCR/TXT_CLEAN.csv
***Removing 3 columns: NO_MUNICIPIO, NU_ANO_CENSO, NO_REGIAO
Loading ../data/interim/2021_NCR/ICG_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2021_NCR/TXR_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO
Loading ../data/interim/2021_NCR/IED_CLEAN.csv
***Removing 4 columns: SG_UF, NU_ANO_CENSO, NO_MUNICIPIO, NO_REGIAO

---------- Start merging 2021_NCR data... ----------
Número de linhas com TXT_FUN_TX_EVASAO_T

In [9]:
# Build final dataset with all years
# years = ["2021_NCR_v2"]
years = [f"{year}_NCR" for year in range(2013, 2022)]


def remove_year(col_name):
    if "PIB_" in col_name or "PERCENTUAL_FAMILIAS_PBF_" in col_name:
        return col_name.rsplit("_", 1)[0]
    return col_name


print("\n\n---------- Start building final dataset... ----------")
final_complete_data = pd.DataFrame()
final_total_total_data = pd.DataFrame()
final_total_total_data_fun = pd.DataFrame()
final_total_total_data_med = pd.DataFrame()


for folder in years:
    print(f"Loading data from {folder}...")

    ############################ final_complete_data ##############################
    year_complete_data = pd.read_csv(
        os.path.join(PROCESSED_DATA_DIR, folder, "complete_data.csv"), delimiter=";"
    )
    year_complete_data.rename(columns=remove_year, inplace=True)
    print(
        f"{folder} add {year_complete_data.shape[0]-1} new rows to final_complete_Data"
    )
    final_complete_data = pd.concat([final_complete_data, year_complete_data])
    ############################# final_total_total_data #############################
    year_data_total_total = pd.read_csv(
        os.path.join(PROCESSED_DATA_DIR, folder, "total_total_data.csv"), delimiter=";"
    )
    year_data_total_total.rename(columns=remove_year, inplace=True)
    print(
        f"{folder} add {year_data_total_total.shape[0]-1} new rows to total_total_data"
    )
    final_total_total_data = pd.concat([final_total_total_data, year_data_total_total])
    ############################ final_total_total_data_fun ##############################
    year_data_total_total_fun = pd.read_csv(
        os.path.join(PROCESSED_DATA_DIR, folder, "total_total_data_fun.csv"),
        delimiter=";",
    )
    print(
        f"{folder} add {year_data_total_total_fun.shape[0]-1} new rows to total_total_data_fun"
    )
    final_total_total_data_fun = pd.concat(
        [final_total_total_data_fun, year_data_total_total_fun]
    )
    ######################### final_total_total_data_med #################################
    year_data_total_total_med = pd.read_csv(
        os.path.join(PROCESSED_DATA_DIR, folder, "total_total_data_med.csv"),
        delimiter=";",
    )
    print(
        f"{folder} add {year_data_total_total_med.shape[0]-1} new rows to total_total_data_fun"
    )
    final_total_total_data_med = pd.concat(
        [final_total_total_data_med, year_data_total_total_med]
    )

_get_df_info(final_complete_data, "final complete data")
_get_df_info(final_total_total_data, "final total total data")
_get_df_info(final_total_total_data_fun, "final total total data fun")
_get_df_info(year_data_total_total_med, "year data total total med")

# Drop useless rows
for df, df_name in [
    (final_complete_data, "final_complete_data"),
    (final_total_total_data, "final_total_total_data"),
    (final_total_total_data_fun, "final_total_total_data_fun"),
    (final_total_total_data_med, "final_total_total_data_med"),
]:
    if "fun" in df_name:
        useless_row = ["TXT_FUN_TX_EVASAO_TOTAL"]
    elif "med" in df_name:
        useless_row = ["TXT_MED_TX_EVASAO_TOTAL"]
    else:
        useless_row = ["TXT_FUN_TX_EVASAO_TOTAL", "TXT_MED_TX_EVASAO_TOTAL"]

    num_useless_rows = df[useless_row].isna().all(axis=1).sum()

    if num_useless_rows > 0:
        print(f">>>>> {df_name} has {num_useless_rows} useless rows of {df.shape[0]}")
        df.dropna(subset=useless_row, how="all", inplace=True)
        print(f">>>>> {df_name} has {df.shape[0]} rows now!")


# Save final data
print("Saving final data...")
final_complete_data.to_csv(
    os.path.join(PROCESSED_DATA_DIR, "final_complete_data.csv"), sep=";", index=False
)
final_total_total_data.to_csv(
    os.path.join(PROCESSED_DATA_DIR, "final_total_total_data.csv"), sep=";", index=False
)
final_total_total_data_fun.to_csv(
    os.path.join(PROCESSED_DATA_DIR, "final_total_total_data_fun.csv"),
    sep=";",
    index=False,
)
year_data_total_total_med.to_csv(
    os.path.join(PROCESSED_DATA_DIR, "final_total_total_data_med.csv"),
    sep=";",
    index=False,
)



---------- Start building final dataset... ----------
Loading data from 2013_NCR...
2013_NCR add 23873 new rows to final_complete_Data
2013_NCR add 5564 new rows to total_total_data
2013_NCR add 5564 new rows to total_total_data_fun
2013_NCR add 5564 new rows to total_total_data_fun
Loading data from 2014_NCR...
2014_NCR add 23874 new rows to final_complete_Data
2014_NCR add 5569 new rows to total_total_data
2014_NCR add 5569 new rows to total_total_data_fun
2014_NCR add 5569 new rows to total_total_data_fun
Loading data from 2015_NCR...
2015_NCR add 23850 new rows to final_complete_Data
2015_NCR add 5569 new rows to total_total_data
2015_NCR add 5569 new rows to total_total_data_fun
2015_NCR add 5569 new rows to total_total_data_fun
Loading data from 2016_NCR...
2016_NCR add 23846 new rows to final_complete_Data
2016_NCR add 5569 new rows to total_total_data
2016_NCR add 5569 new rows to total_total_data_fun
2016_NCR add 5569 new rows to total_total_data_fun
Loading data from 2017_N